# Use the Neptune API to log your first run

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/hello-neptune/notebooks/hello_neptune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>
<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/how-to-guides/hello-neptune/notebooks/hello_neptune.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://app.neptune.ai/o/common/org/quickstarts/runs/table?viewId=d48562e1-a494-4fd0-b3bb-078240516a4f"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>
<a target="_blank" href="https://docs.neptune.ai/usage/quickstart/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Install neptune
* Connect Neptune to your code and create the first run
* Log metadata to Neptune and explore them in the Neptune app
* Download logged metadata from Neptune

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune mnist

## Initialize Neptune and create new run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    project="common/quickstarts",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
    source_files="hello_neptune.ipynb",  # does not work in Colab
    tags=["quickstart", "notebook"],
    dependencies="requirements.txt",  # pass the path to the requirements.txt file
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

You'll use the `run` object you just created to log metadata. You'll see the metadata appear in the app.

## Log metrics during training
Log metrics or losses under a name of your choice. You can log one or multiple values.

In [ ]:
from random import random

# log single value
run["seed"] = 0.42

# log series of values
epochs = 10
offset = random() / 5

for epoch in range(epochs):
    acc = 1 - 2**-epoch - random() / (epoch + 1) - offset
    loss = 2**-epoch + random() / (epoch + 1) + offset

    run["accuracy"].append(acc)
    run["loss"].append(loss)

This logs three metrics:
* `seed` with just one value
* `accuracy` with 10 values
* `loss` with 10 values

## Log single and series of images
You can log images either as single files or as a series.
Read more about logging and viewing images in the [docs](https://docs.neptune.ai/logging/images/).

In [ ]:
# Download sample image
! curl -O https://neptune.ai/wp-content/uploads/2023/06/Lenna_test_image.png

In [ ]:
# Upload single image to Neptune
run["single_image"].upload("Lenna_test_image.png")  # You can upload native images as-is

In [ ]:
# Load MNIST dataset
import mnist

train_images = mnist.train_images()
train_labels = mnist.train_labels()

In [ ]:
# Upload a series of images to Neptune
from neptune.types import File

for i in range(10):
    run["image_series"].append(
        File.as_image(
            train_images[i] / 255
        ),  # You can upload arrays as images using Neptune's File.as_image() method
        name=f"{train_labels[i]}",
    )

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
# Save the run ID to resume the run later
run_id = run["sys/id"].fetch()

In [ ]:
run.stop()

## Explore the run in the Neptune app

* Uploaded source files are present in the **Source code** dashboard
* The graphs of the `accuracy` and `loss` fields are displayed in the **Charts** dashboard
* You can view your system hardware consumption in the **Monitoring** dashboard
* The images logged are in the **Images** dashboard

## Download logged metadata from Neptune
Everything logged to Neptune can be downloaded, both from the [web app](https://docs.neptune.ai/usage/downloading_metadata/) and [using the API](https://docs.neptune.ai/usage/querying_metadata/).

### Re-initialize an already logged run
You just need to know the project which the run was logged to, and the [run ID](https://docs.neptune.ai/usage/getting_id/).

In [ ]:
run = neptune.init_run(
    project="common/quickstarts",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    with_id=run_id,  # ID of the run you want to re-initialize
    mode="read-only",  # To prevent accidental overwrite of already logged data
)

### Download metadata from re-initialized run

In [ ]:
run["seed"].fetch()

In [ ]:
run["accuracy"].fetch_values()

In [ ]:
run["single_image"].download("downloaded_single_image.png")
# The image is now downloaded as "downloaded_single_image.png"

In [ ]:
run["image_series"].download("downloaded_image_series")
# The images are downloaded in the "downloaded_image_series" folder

### Stop the run

In [ ]:
run.stop()

## What's next

* See [other Neptune examples](https://docs.neptune.ai/getting-started/examples)
* Learn what [metadata you can log to Neptune](https://docs.neptune.ai/logging/what_you_can_log/)